In [1]:
# Check the installation of Torch and the use of GPU
import torch, torchvision
print(torch.__version__, torch.cuda.is_available())

# Check the installation of MMAction2
import mmaction
print(mmaction.__version__)

# Check the installation of the MMCV
from mmcv.ops import get_compiling_cuda_version, get_compiler_version
print(get_compiling_cuda_version())
print(get_compiler_version())

1.10.0+cu113 True
0.15.0
11.3
MSVC 192930137


In [2]:
from mmaction.apis import inference_recognizer, init_recognizer

# Select the configuration file for TSN
config = 'configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py'
# Load the checkpoint file downloaded above
checkpoint =  'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# Initialize the model
print(config)
#model = init_recognizer(config, checkpoint, device='cuda:1')

configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py


In [3]:
from mmcv.runner import set_random_seed
# Obtain the cfg configuration file corresponding to TSN
from mmcv import Config
cfg = Config.fromfile('./configs/recognition/swin/swin_base_patch244_window877_kinetics400_1k.py')
# Modify the dataset type and individual file paths
cfg.dataset_type = 'VideoDataset'
cfg.data_root = './video_data/train_vidieoB/'
cfg.data_root_val = './video_data/val_videoB/'
cfg.ann_file_train = './video_data/train.txt'
cfg.ann_file_val = './video_data/val2.csv'
cfg.ann_file_test = './video_data/val2.csv'


cfg.data.test.type = 'VideoDataset'
cfg.data.test.ann_file = './video_data/val2.csv'
cfg.data.test.data_prefix = './video_data/val_videoB/'

cfg.data.train.type = 'VideoDataset'
cfg.data.train.ann_file = './video_data/train.txt'
cfg.data.train.data_prefix = './video_data/train_videoB/'
cfg.data.train.data_prefix = './video_data/train_videoB/'

cfg.data.val.type = 'VideoDataset'
cfg.data.val.ann_file = './video_data/val2.csv'
cfg.data.val.data_prefix = './video_data/val_video/'

# This is used to confirm that omnisource training is used
cfg.setdefault('omnisource', False)
# The number of categories in the modified cls_head is 3
cfg.model.cls_head.num_classes = 3
# Use a pre-trained TSN model
cfg.load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
# Set up a working directory
cfg.work_dir = './tutorial_exps'

# Since it is a single-card training, the corresponding LR is modified
#cfg.data.videos_per_gpu = cfg.data.videos_per_gpu // 16
cfg.data.videos_per_gpu = 2
cfg.optimizer.lr = cfg.optimizer.lr / 8 / 16
#Set up epoches
cfg.total_epochs = 20
# Set the save point interval to reduce the consumption of storage space
cfg.checkpoint_config.interval = 10
# Set the log printing interval to reduce the printing time
cfg.log_config.interval = 5

# Fixed random seeds make the results reproducible
cfg.seed = 40
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1,2)

# Print all configuration parameters
print(f'Config:\n{cfg.pretty_text}')

Config:
model = dict(
    type='Recognizer3D',
    backbone=dict(
        type='SwinTransformer3D',
        patch_size=(2, 4, 4),
        embed_dim=128,
        depths=[2, 2, 18, 2],
        num_heads=[4, 8, 16, 32],
        window_size=(8, 7, 7),
        mlp_ratio=4.0,
        qkv_bias=True,
        qk_scale=None,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.3,
        patch_norm=True),
    cls_head=dict(
        type='I3DHead',
        in_channels=1024,
        num_classes=3,
        spatial_type='avg',
        dropout_ratio=0.5),
    test_cfg=dict(average_clips='prob', max_testing_views=4))
checkpoint_config = dict(interval=10)
log_config = dict(interval=5, hooks=[dict(type='TextLoggerHook')])
dist_params = dict(backend='nccl')
log_level = 'INFO'
load_from = 'checkpoints/swin_base_patch244_window877_kinetics400_1k.pth'
resume_from = None
workflow = [('train', 1)]
dataset_type = 'VideoDataset'
data_root = './video_data/train_vidieoB/'
data_root_val = '.

In [4]:
import os.path as osp
import os
def clean():
    # Get a list of all files in the current directory
    files_in_directory = os.listdir()
    filtered_files = [file for file in files_in_directory if file.endswith(".pt")]
    # Loop through the list and remove each file
    for file in filtered_files:
        os.remove(file)
def cleancsv(directory):
    # Specify the directory where you want to clear the CSV file
    # Iterate through all the files in the directory
    for filename in os.listdir(directory):
        # Check if the file extension is '.csv'
        if filename.endswith('.csv'):
            # Construct the complete file path
            file_path = os.path.join(directory, filename)
            # Delete the file
            os.remove(file_path)

In [5]:
cfg.ann_file_val = './video_data/all-B.csv'
cfg.ann_file_test = './video_data/all-B.csv'
cfg.data.test.ann_file = './video_data/all-B.csv'
cfg.data.test.data_prefix = './video_data/val_videoB/'
cfg.data.val.ann_file = './video_data/all-B.csv'
cfg.data.val.data_prefix = './video_data/val_videoB/'

In [6]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model,train_model_1
cfg.data.val.type = 'VideoDataset'
# Build a test dataset
model = torch.load('Swin-SPL-clean-91.pth')
path = "./save/"
cleancsv(path)
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[1])
outputs = single_gpu_test(model, data_loader)

# The trained recognition model was evaluated on the test set
eval_config = cfg.evaluation
#eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

[                                                  ] 0/188, elapsed: 0s, ETA:

KeyboardInterrupt: 

In [ ]:
from mmaction.apis import single_gpu_test
from mmaction.datasets import build_dataloader
from mmcv.parallel import MMDataParallel
from mmaction.datasets import build_dataset
from mmaction.models import build_model
from mmaction.apis import train_model,train_model_1
cfg.data.val.type = 'VideoDataset'
# Build a test dataset
model = torch.load('Swin-SPL-hard.pth')
path = "./save/"
cleancsv(path)
dataset = build_dataset(cfg.data.val, dict(test_mode=True))
data_loader = build_dataloader(
        dataset,
        videos_per_gpu=1,
        workers_per_gpu=cfg.data.workers_per_gpu,
        dist=False,
        shuffle=False)
model = MMDataParallel(model, device_ids=[1])
outputs = single_gpu_test(model, data_loader)

# The trained recognition model was evaluated on the test set
eval_config = cfg.evaluation
#eval_config.pop('interval')#interval=5
eval_res = dataset.evaluate(outputs, **eval_config)
for name, val in eval_res.items():
    print(f'{name}: {val:.04f}')

In [7]:
import os
from time import localtime
import pandas as pd
from operator import itemgetter
def readfile(directory):
    # A list of stored files and when they were modified
    files_with_mtime = []
    # Iterate through all the files in the directory
    for filename in os.listdir(directory):
        # Construct the complete file path
        file_path = os.path.join(directory, filename)
        # Make sure it's a file and not a directory
        if os.path.isfile(file_path):
            # Gets the modification time of the file
            modification_time = os.path.getmtime(file_path)
            # Convert the timestamp to local time
            local_time = localtime(modification_time)
            # Add the file name and converted local time to the list
            files_with_mtime.append((filename, local_time))
    # Filter out the CSV file and sort it by modification time
    csv_files_sorted = sorted(files_with_mtime, key=itemgetter(1))
    # Store a list of read DataFrames
    dataframes = []
    # Traverse the sorted CSV file
    for filename, mtime in csv_files_sorted:
        # Construct the complete file path
        file_path = os.path.join(directory, filename)
        # Read the CSV file
        df = pd.read_csv(file_path)
        # Add the read DataFrame to the list
        dataframes.append((df, mtime))
        # Prints the read file name and the time when it was modified
        print(f"the file has been read：{filename}, modify time：{mtime}")
        # If required, the read data can be further processed
        # For example, merge all DataFrames
        combined_df = pd.concat([df for df, _ in dataframes], ignore_index=True)
    return combined_df

In [8]:
def process(combined_df):
    df_cleaned = combined_df.applymap(lambda x: x.replace("tensor(", "").replace(")", "") if isinstance(x, str) else x)
    return df_cleaned
def str_to_float(x):
    # Check if x is a string, and if so, try to convert to floating-point
    if isinstance(x, str):
        try:
            return float(x)
        except ValueError:
            return x  # If the conversion fails, the original value is returned
    return x

In [9]:
# 指定目录路径
directory = './save2/all-train-c/'
combined_df = readfile(directory)
df_cleaned = process(combined_df)
all_train_c = df_cleaned.applymap(str_to_float)

已读取文件：8_52_4.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=5, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_52_6.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=6, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_52_8.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=8, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_52_9.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=10, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_52_11.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=12, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_52_13.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=13, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_52_15.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=52, tm_sec=15, tm_wday=2, tm_yday=157, tm_i

已读取文件：8_53_41.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=53, tm_sec=42, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_53_43.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=53, tm_sec=43, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_53_45.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=53, tm_sec=45, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_53_46.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=53, tm_sec=47, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_53_48.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=53, tm_sec=49, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_53_50.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=53, tm_sec=50, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_53_51.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=53, tm_sec=52, tm_wday=2, tm_yday=15

已读取文件：8_55_21.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=55, tm_sec=21, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_55_23.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=55, tm_sec=23, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_55_24.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=55, tm_sec=25, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_55_26.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=55, tm_sec=27, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_55_28.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=55, tm_sec=28, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_55_29.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=55, tm_sec=30, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：8_55_31.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=8, tm_min=55, tm_sec=32, tm_wday=2, tm_yday=15

In [10]:
# 指定目录路径
directory = './save2/all-train-n/'
combined_df = readfile(directory)
df_cleaned = process(combined_df)
all_train_n = df_cleaned.applymap(str_to_float)

已读取文件：9_7_52.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=7, tm_sec=53, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_7_54.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=7, tm_sec=55, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_7_56.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=7, tm_sec=56, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_7_58.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=7, tm_sec=58, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_7_59.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=8, tm_sec=0, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_8_1.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=8, tm_sec=2, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_8_3.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=8, tm_sec=4, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文

已读取文件：9_9_39.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=9, tm_sec=40, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_9_41.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=9, tm_sec=42, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_9_43.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=9, tm_sec=44, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_9_45.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=9, tm_sec=45, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_9_46.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=9, tm_sec=47, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_9_48.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=9, tm_sec=49, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_9_50.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=9, tm_sec=51, tm_wday=2, tm_yday=157, tm_isdst=0)

已读取文件：9_11_19.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=11, tm_sec=20, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_11_21.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=11, tm_sec=22, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_11_23.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=11, tm_sec=23, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_11_25.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=11, tm_sec=25, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_11_26.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=11, tm_sec=27, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_11_28.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=11, tm_sec=29, tm_wday=2, tm_yday=157, tm_isdst=0)
已读取文件：9_11_30.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=5, tm_hour=9, tm_min=11, tm_sec=30, tm_wday=2, tm_yday=15

In [11]:
# 指定目录路径
directory = './save/datasetB-clean/'
combined_df = readfile(directory)
df_cleaned = process(combined_df)
all_sample_c = df_cleaned.applymap(str_to_float)

已读取文件：11_39_30.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=39, tm_sec=30, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_39_31.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=39, tm_sec=31, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_39_32.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=39, tm_sec=32, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_39_34.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=39, tm_sec=34, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_39_35.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=39, tm_sec=35, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_39_36.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=39, tm_sec=37, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_39_38.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=39, tm_sec=38, 

已读取文件：11_40_45.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=40, tm_sec=46, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_40_47.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=40, tm_sec=47, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_40_48.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=40, tm_sec=48, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_40_49.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=40, tm_sec=50, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_40_51.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=40, tm_sec=51, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_40_52.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=40, tm_sec=52, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_40_53.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=40, tm_sec=54, 

已读取文件：11_42_1.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=42, tm_sec=1, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_42_2.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=42, tm_sec=2, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_42_4.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=42, tm_sec=4, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_42_5.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=42, tm_sec=5, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_42_6.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=42, tm_sec=6, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_42_7.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=42, tm_sec=8, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_42_9.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=42, tm_sec=9, tm_wday=2, tm_

已读取文件：11_43_17.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=43, tm_sec=17, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_43_18.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=43, tm_sec=18, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_43_19.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=43, tm_sec=19, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_43_21.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=43, tm_sec=21, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_43_22.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=43, tm_sec=22, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_43_23.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=43, tm_sec=23, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：11_43_24.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=11, tm_min=43, tm_sec=25, 

In [12]:
# 指定目录路径
directory = './save/datasetB-hard/'
combined_df = readfile(directory)
df_cleaned = process(combined_df)
all_sample_n = df_cleaned.applymap(str_to_float)

已读取文件：16_2_12.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=2, tm_sec=12, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_2_13.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=2, tm_sec=13, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_2_14.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=2, tm_sec=14, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_2_16.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=2, tm_sec=16, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_2_17.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=2, tm_sec=17, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_2_18.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=2, tm_sec=19, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_2_20.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=2, tm_sec=20, tm_wday=2, tm_

已读取文件：16_3_28.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=3, tm_sec=28, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_3_29.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=3, tm_sec=29, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_3_30.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=3, tm_sec=30, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_3_32.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=3, tm_sec=32, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_3_33.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=3, tm_sec=33, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_3_34.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=3, tm_sec=34, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_3_35.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=3, tm_sec=36, tm_wday=2, tm_

已读取文件：16_4_43.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=4, tm_sec=43, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_4_44.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=4, tm_sec=45, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_4_46.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=4, tm_sec=46, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_4_47.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=4, tm_sec=47, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_4_48.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=4, tm_sec=49, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_4_50.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=4, tm_sec=50, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_4_51.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=4, tm_sec=51, tm_wday=2, tm_

已读取文件：16_5_59.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=5, tm_sec=59, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_6_0.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=6, tm_sec=0, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_6_1.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=6, tm_sec=2, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_6_3.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=6, tm_sec=3, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_6_4.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=6, tm_sec=4, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_6_5.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=6, tm_sec=5, tm_wday=2, tm_yday=164, tm_isdst=0)
已读取文件：16_6_7.csv, 修改时间：time.struct_time(tm_year=2024, tm_mon=6, tm_mday=12, tm_hour=16, tm_min=6, tm_sec=7, tm_wday=2, tm_yday=164, tm

In [13]:
all_train_c *= 1.2
all_train_n *= 0.8
train = pd.concat([all_train_c, all_train_n], axis=1)
all_sample_c *= 1.2
all_sample_n *= 0.8
val = pd.concat([all_sample_c, all_sample_n], axis=1)

In [14]:
# Specify the directory path and file name
import pandas as pd
# Specify the path to .txt file
file_path = './video_data/all-train.csv'
# Determine the separator (e.g., comma ',', tab 't', space '', etc.)
# The delimiter is assumed to be a tab
delimiter = ' '
Y_train = pd.read_csv(file_path, delimiter=delimiter, header=None)
file_path = './video_data/all-B.csv'
# Determine the separator (e.g., comma ',', tab 't', space '', etc.)
# The delimiter is assumed to be a tab
delimiter = ' '
Y_val = pd.read_csv(file_path, delimiter=delimiter, header=None)

In [15]:
Y_train = Y_train.iloc[:, -1]
Y_val = Y_val.iloc[:, -1]

In [16]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Initialize the random forest model
# n_estimators is the number of trees in the forest, and max_depth is the maximum depth of the trees
rf = RandomForestClassifier(n_estimators=100, max_depth=10)

# Train the model
rf.fit(train, Y_train)

# Predict the test set
y_pred = rf.predict(val)

# Evaluate the model
accuracy = accuracy_score(Y_val, y_pred)
print(f"model accuracy: {accuracy}")

模型准确率: 0.7340425531914894


In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import ElasticNet

# Define the model
models = {
    'Random Forest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM': SVC(probability=True, random_state=42),
    'Logistic Regression': LogisticRegression(max_iter=200),
    'MLP': MLPClassifier(max_iter=100),
    #'Elastic Net': ElasticNet()
}

# Evaluate the model and print the results
for name, model in models.items():
    # Train the model
    model.fit(train, Y_train)
    
    # Predict the test set
    y_pred = model.predict(val)
    
    # Calculate the accuracy
    accuracy = accuracy_score(Y_val, y_pred)
    print(f"{name} accuracy: {accuracy:.2f}")

Random Forest 的准确率: 0.77
SVM 的准确率: 0.80
Logistic Regression 的准确率: 0.73
MLP 的准确率: 0.73


In [18]:
rf = SVC(probability=True, random_state=42)

# Train the model
rf.fit(train, Y_train)

# Predict the test set
y_pred = rf.predict(val)

# Evaluate the model
accuracy = accuracy_score(Y_val, y_pred)
print(f"Model accuracy: {accuracy}")

模型准确率: 0.8031914893617021


In [23]:
Y_val.values

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [20]:
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 1, 2, 1, 1, 1, 1, 2, 1,
       1, 1, 1, 1, 0, 0, 1, 2, 1, 1, 1, 1, 1, 0, 1, 0, 1, 2, 2, 0, 1, 0,
       1, 0, 0, 1, 2, 0, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 1, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0,
       2, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 0, 1, 1,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int64)

In [21]:
y_pred = pd.DataFrame(y_pred)
y_pred.to_csv('output.csv', index=False)

In [24]:
from sklearn.metrics import classification_report
import numpy as np
print(classification_report(Y_val, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.93      0.84        43
           1       0.81      0.70      0.75        77
           2       0.83      0.84      0.83        68

    accuracy                           0.80       188
   macro avg       0.80      0.82      0.81       188
weighted avg       0.80      0.80      0.80       188

